# FINAL CODE STEPS FOR SEARCH ENGINE PROJECT

## Step 1: Reading the data from the database

### Part 1

In [ ]:
import sqlite3
conn = sqlite3.connect("Data/eng_subtitles_database.db")

In [ ]:
import pandas as pd
query = 'SELECT * FROM zipfiles'
df = pd.read_sql_query(query, conn)
conn.close()
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [ ]:
df.shape

(82498, 3)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.9+ MB


In [ ]:
from tqdm import tqdm, tqdm_notebook
tqdm.pandas()

In [ ]:
import zipfile
import io
def decomp_decode(data):
    with zipfile.ZipFile(io.BytesIO(data)) as zip_file:
        file_list = zip_file.namelist()
        first_file = file_list[0]
        decompressed_data = zip_file.read(first_file)
    return decompressed_data.decode('latin-1')

In [ ]:
df['content'] = df['content'].progress_apply(lambda x : decomp_decode(x))

100%|██████████████████████████████████████████████████████████████████████████| 82498/82498 [00:26<00:00, 3125.53it/s]


In [ ]:
df['content'].head()

0    1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...
1    1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther...
2    1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'...
3    1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an...
4    ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch...
Name: content, dtype: object

In [ ]:
import re

def clean_data(data):
    data = re.sub("\d{2}:\d{2}:\d{2},\d{3}\s-->\s\d{2}:\d{2}:\d{2},\d{3}"," ",  data)
    data = re.sub(r'\n?\d+\r', "", data)
    data = re.sub('\r|\n', "", data)
    data = re.sub('<i>|</i>', "", data)
    data = re.sub("(?:www\.)osdb\.link\/[\w\d]+|www\.OpenSubtitles\.org|osdb\.link\/ext|api\.OpenSubtitles\.org|OpenSubtitles\.com", " ",data)
    data = data.lower()
    return data

In [ ]:
df['content'] = df['content'].progress_apply(lambda x: clean_data(x))

100%|███████████████████████████████████████████████████████████████████████████| 82498/82498 [04:54<00:00, 280.38it/s]


In [ ]:
df['num'] = df['num'].apply(lambda x : str(x)) 

In [ ]:
#!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model_name = 'paraphrase-MiniLM-L3-v2' 
model = SentenceTransformer(model_name, device='cuda')

def semantic_chunking(document, similarity_threshold=0.9):
    sentences = document.split('.')
    chunks = []
    current_chunk = sentences[0]
    sentence_embeddings = model.encode(sentences)
    for i in range(1, len(sentences)):
        similarity_score = np.dot(sentence_embeddings[i], sentence_embeddings[i-1]) / (np.linalg.norm(sentence_embeddings[i]) * np.linalg.norm(sentence_embeddings[i-1]))
        if similarity_score >= similarity_threshold:
            current_chunk += '.' + sentences[i]
        else:
            chunks.append(current_chunk)
            current_chunk = sentences[i]

    chunks.append(current_chunk)

    return chunks

In [ ]:
temp_1 = pd.DataFrame()
temp_2 = pd.DataFrame()

In [ ]:
temp_1['num']=df['num'][:30000]

In [ ]:
temp_2['num']=df['num'][30000:]

In [ ]:
from joblib import Parallel, delayed
import time
start = time.time()
temp_1['chunks'] = Parallel(n_jobs=-1)(delayed(semantic_chunking)(item) for item in df['content'].values[:30000])
end=time.time()
print(f"Total time in seconds = {end-start}")

Total time in seconds = 4418.998434782028


In [ ]:
temp_1.to_json("database.json") 

In [ ]:
from joblib import Parallel, delayed
import time
start = time.time()
temp_2['chunks'] = Parallel(n_jobs=-1)(delayed(semantic_chunking)(item) for item in df['content'].values[30000:])
end=time.time()
print(f"Total time in seconds = {end-start}")

Total time in seconds = 8644.886986494064


In [ ]:
temp_2.to_json("database_p2.json")

In [ ]:
import json

json_file_path = "database.json" 
with open(json_file_path, 'r') as f:
  data = json.load(f)

df = pd.DataFrame(data)

In [ ]:
def indexer(item):
    index=[]
    temp=int(df[df['num']==item].index[0])
    for j in range(len(df['chunks'].iloc[temp])):
        index.append(item+"-"+str(j))
    return index

In [ ]:
df['num_list'] = df['num'].apply(lambda x : indexer(x))

In [ ]:
from sentence_transformers import SentenceTransformer

model_name = 'paraphrase-MiniLM-L3-v2' #all-MiniLM-L6-v2
model = SentenceTransformer(model_name, device='cuda')

In [ ]:
def embedding_gen(data):
  return model.encode(data).tolist()

In [ ]:
df['embeddings'] = Parallel(n_jobs=-1)(delayed(embedding_gen)(item) for item in df['chunks'].values)

In [ ]:
import chromadb
client = chromadb.PersistentClient(path="E://search_engine_db")
collection = client.get_or_create_collection(name="search_engine", metadata={"hnsw:space": "cosine"})
collection_2 = client.get_or_create_collection(name="search_engine_FileName", metadata={"hnsw:space": "cosine"})

In [ ]:
def add_func_v1():
    for i in range(df.shape[0]): 
        collection_2.add(
            documents=[df['name'].iloc[i]],
            embeddings=[[1,2,34,45]], 
            ids=[df['num'].iloc[i]] 
        )

In [ ]:
def add_func_v2():
    for i in range(df.shape[0]):
        collection.add(
            documents=df['chunks'].iloc[i], 
            embeddings=df['embeddings'].iloc[i], 
            ids=df['num_list'].iloc[i] 
        )

In [ ]:
%time add_func_v1()

In [ ]:
%time add_func_v2()

### Streamlit

In [ ]:
import re
import chromadb
from sentence_transformers import  SentenceTransformer
import streamlit as st

### Initializing chromaDB


In [ ]:
client = chromadb.PersistentClient(path="E://search_engine_db") 
collection = client.get_collection(name="search_engine") 
collection_name = client.get_collection(name="search_engine_FileName")
model_name="paraphrase-MiniLM-L3-V2"
model = SentenceTransformer(model_name, device="cuda")

In [ ]:
def clean_data(data):
    data = re.sub("\d{2}:\d{2}:\d{2},\d{3}\s-->\s\d{2}:\d{2}:\d{2},\d{3}"," ",  data)
    data = re.sub(r'\n?\d+\r', "", data)
    data = re.sub('\r|\n', "", data)
    data = re.sub('<i>|</i>', "", data)
    data = re.sub("(?:www\.)osdb\.link\/[\w\d]+|www\.OpenSubtitles\.org|osdb\.link\/ext|api\.OpenSubtitles\.org|OpenSubtitles\.com", " ",data)
    data = data.lower()
    return data

In [ ]:
def extract_id(id_list):
    new_id_list=[]
    for item in id_list:
        match = re.match(r'^(\d+)', item)
        if match:
            extracted_number = match.group(1)
            new_id_list.append(extracted_number)
    return new_id_list

### Creating the web application

In [ ]:
st.header("Enhancing Search Engine Relevance for Video Subtitles")
search_query=st.text_input("Enter a dialogue to search....")
if st.button("Search")==True:

    st.subheader("Relevant Subtitle Files")
    search_query=clean_data(search_query)
    query_embed = model.encode(search_query).tolist()

    search_results=collection.query(query_embeddings=query_embed, n_results=10)
    id_list = search_results['ids'][0]

    id_list = extract_id(id_list)
    print(id_list)
    for id in id_list:
        file_name = collection_name.get(ids=f"{id}")["documents"][0]
        st.markdown(f"[{file_name}](https://www.opensubtitles.org/en/subtitles/{id})")
